In [1]:
import os
import re
from PIL import Image, ImageDraw, ImageFont
import cairosvg
from io import BytesIO

In [2]:
BASE_DIR = "attention_analysis/"
# OUTPUT_FILE = "perhead_attention_comparison_grid.png"
OUTPUT_FILE = "heatmap_attention_comparison_grid.png"
COLUMNS = 6
ROWS = 5
LABEL_HEIGHT = 40  # 为每张图留出写“Layer X”的空间

In [3]:
def get_layer_num(dir_name):
    """从文件夹名提取数字，确保按 0, 1, 2...29 排序而非 0, 1, 10..."""
    match = re.search(r'layer(\d+)', dir_name)
    return int(match.group(1)) if match else -1

In [4]:
def create_grid():
    # 1. 获取并排序所有层目录
    layer_dirs = [d for d in os.listdir(BASE_DIR) if d.startswith('layer')]
    layer_dirs.sort(key=get_layer_num)
    
    processed_images = []
    cell_width = 0
    cell_height = 0

    print("开始渲染 SVG 并添加标注...")
    
    for folder in layer_dirs:
        layer_idx = get_layer_num(folder)
        file_path = os.path.join(BASE_DIR, folder, f"{folder}_2d_heatmap_all_heads.svg")
        # file_path = os.path.join(BASE_DIR, folder, f"{folder}_perhead_grid.svg")
        
        if not os.path.exists(file_path):
            continue

        # 将 SVG 转为 PNG 图片
        png_data = cairosvg.svg2png(url=file_path)
        img = Image.open(BytesIO(png_data)).convert("RGB")
        
        # 统一尺寸（以第一张图为准）
        if cell_width == 0:
            cell_width = img.width
            cell_height = img.height

        # 创建一个带标题的单元格画布
        cell = Image.new("RGB", (cell_width, cell_height + LABEL_HEIGHT), (255, 255, 255))
        cell.paste(img, (0, LABEL_HEIGHT))
        
        # 绘制文字标签
        draw = ImageDraw.Draw(cell)
        # 如果报错，可以移除 font 参数使用默认字体
        try:
            font = ImageFont.truetype("arial.ttf", 24)
        except:
            font = ImageFont.load_default()
            
        draw.text((10, 5), f"Layer {layer_idx}", fill=(0, 0, 0), font=font)
        
        processed_images.append(cell)

    if not processed_images:
        print("未找到图片，请检查路径。")
        return

    # 2. 创建最终的大画布
    grid_width = cell_width * COLUMNS
    grid_height = (cell_height + LABEL_HEIGHT) * ROWS
    final_image = Image.new("RGB", (grid_width, grid_height), (255, 255, 255))

    # 3. 拼接
    for i, img in enumerate(processed_images):
        x = (i % COLUMNS) * cell_width
        y = (i // COLUMNS) * (cell_height + LABEL_HEIGHT)
        final_image.paste(img, (x, y))

    # 4. 保存
    final_image.save(OUTPUT_FILE)
    print(f"✅ 成功！对比图已生成：{OUTPUT_FILE}")
    print(f"尺寸: {grid_width}x{grid_height}")

In [5]:
create_grid()

开始渲染 SVG 并添加标注...
✅ 成功！对比图已生成：heatmap_attention_comparison_grid.png
尺寸: 9282x6090
